In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyodbc sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install bson

  Created wheel for bson: filename=bson-0.5.10-py3-none-any.whl size=11978 sha256=eda64bb5685650e575f9968140d834e2009b8e8154b8d42caacd483d15c700d9
  Stored in directory: /home/leonardo/.cache/pip/wheels/92/1b/66/5a288fdf87a97b2f7e2cbdc387a431e60ed298b11003692055
Successfully built bson
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pymongo


     |████████████████████████████████| 930 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 307 kB 17.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import random
import re
import uuid

from sqlalchemy import create_engine, text, MetaData, Table, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker
from bson import ObjectId

### Inicializando uma instância do google chrome

In [12]:
# A classe service é usada para iniciar uma instancia do chrome webdriver
service = Service()

# webdriver.Chromeoptions é usado para definir a preferência para o brower do chrome
options = webdriver.ChromeOptions()

# Inicia-se a instância do chrome WebDriver com as definidas 'options' e 'service'
driver = webdriver.Chrome(service=service, options=options)

In [13]:
url = "https://www.espn.com.br/nba/classificacao/_/grupo/liga"

driver.get(url)

### Raspagem dos Times e Links


In [14]:
# Encontra todos os elementos com a tag <a>
all_teams = []

selector = 'a[data-clubhouse-uid^="s:40~l:46~t:"]'
teams = driver.find_elements(By.CSS_SELECTOR, selector)

# Filtra os elementos para remover espaços em branco ou elementos sem texto
all_teams = [team for team in teams if team.text.strip()]

print(f"Encontrados {len(all_teams)} times")

for team in all_teams:
  print(team.text)

team_names = [team.text for team in all_teams]

links = [link.get_attribute('href') for link in all_teams]
print(links)


Encontrados 30 times
Cleveland Cavaliers
Oklahoma City Thunder
Boston Celtics
New York Knicks
Denver Nuggets
Houston Rockets
Memphis Grizzlies
Los Angeles Lakers
Golden State Warriors
Minnesota Timberwolves
Milwaukee Bucks
Indiana Pacers
Detroit Pistons
LA Clippers
Sacramento Kings
Dallas Mavericks
Atlanta Hawks
Phoenix Suns
Orlando Magic
Miami Heat
Chicago Bulls
Portland Trail Blazers
San Antonio Spurs
Toronto Raptors
Brooklyn Nets
Philadelphia 76ers
New Orleans Pelicans
Charlotte Hornets
Utah Jazz
Washington Wizards
['https://www.espn.com.br/nba/time/_/nome/cle/cleveland-cavaliers', 'https://www.espn.com.br/nba/time/_/nome/okc/oklahoma-city-thunder', 'https://www.espn.com.br/nba/time/_/nome/bos/boston-celtics', 'https://www.espn.com.br/nba/time/_/nome/ny/new-york-knicks', 'https://www.espn.com.br/nba/time/_/nome/den/denver-nuggets', 'https://www.espn.com.br/nba/time/_/nome/hou/houston-rockets', 'https://www.espn.com.br/nba/time/_/nome/mem/memphis-grizzlies', 'https://www.espn.com.br/

### Raspagem dos jogadores


In [15]:
list_teams = []
list_players = []

for i, link in enumerate(links):
  driver.get(links[i])
  selector = 'a[data-track-nav_item="elenco"]'
  driver.find_element(By.CSS_SELECTOR, selector).click()

  ## Delay de 2 a 5 segundos
  time.sleep(random.uniform(2, 5))
  team_data = {}
  
  ## Pegando técnico e link da imagem
  technical = driver.find_element(By.CLASS_NAME, "TableDetails__Paragraph").text.replace("Nome do técnico", "").strip()
  image_link = driver.find_element(By.CLASS_NAME, "Logo__xxl").get_attribute('src')
  
  team_data = {
    "_id": ObjectId(),
    "time": team_names[i],
    "technical": technical,
    "image_link": image_link,
  }

  list_teams.append(team_data)

  # Encontra a tabela (sem classes)
  table = driver.find_element(By.TAG_NAME, "table")

  # Encontra todas as linhas (tr) da tabela
  rows = table.find_elements(By.TAG_NAME, "tr")

  # Itera pelas linhas da tabela (ignorando o cabeçalho, se existir)
  for row in rows[1:]: # Ignora o cabeçalho (primeira linha)
    player_data = {}  
    cells = row.find_elements(By.TAG_NAME, "td")  # Encontra todas as células

    if cells:  # Verifica se a linha contém células (se não for uma linha vazia)
        # Pegue a imagem
        image_player = row.find_element(By.TAG_NAME, "img").get_attribute("alt") 

        # Extrai o texto das células da linha
        player_info = [cell.text.strip() for cell in cells]

        # Atribui os dados ao dicionário de jogador
        if len(player_info) >= 7:
          name = re.match(r"\D+",player_info[1]).group(0) if player_info[1] else None

          number = re.search(r"\d+",player_info[1]).group(0) if re.search(r"\d+", player_info[1]) else None

          player_data = {
            "image": image_player,
            "name": name,
            "number": number,
            "position": player_info[2],
            "age": player_info[3],
            "height": player_info[4],
            "weight": player_info[5],
            "university": player_info[6],
            "salary": player_info[7],
            "team_id": team_data["_id"],
          }

          # Adiciona o jogador à lista de jogadores
          list_players.append(player_data)


In [25]:
for team in list_teams:
  print(team)


{'_id': ObjectId('67d5ef17e936e1912e2b2dd4'), 'time': 'Cleveland Cavaliers', 'technical': 'Kenny Atkinson', 'image_link': 'https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/cle.png&h=200&w=200'}
{'_id': ObjectId('67d5ef31e936e1912e2b2dd5'), 'time': 'Oklahoma City Thunder', 'technical': 'Mark Daigneault', 'image_link': 'https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/okc.png&h=200&w=200'}
{'_id': ObjectId('67d5ef4be936e1912e2b2dd6'), 'time': 'Boston Celtics', 'technical': 'Joe Mazzulla', 'image_link': 'https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/bos.png&h=200&w=200'}
{'_id': ObjectId('67d5ef67e936e1912e2b2dd7'), 'time': 'New York Knicks', 'technical': 'Tom Thibodeau', 'image_link': 'https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/ny.png&h=200&w=200'}
{'_id': ObjectId('67d5ef83e936e1912e2b2dd8'), 'time': 'Denver Nuggets', 'technical': 'Michael Malone', 'image_link': 'https://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/den.png&h=200&w=200'}


In [26]:
for player in list_players:
  print(player)

{'image': 'https://a.espncdn.com/i/headshots/nba/players/full/4066328.png', 'name': 'Jarrett Allen', 'number': '31', 'position': 'C', 'age': '26', 'height': '2.06 m', 'weight': '110 kg', 'university': 'Texas', 'salary': '$20,000,000', 'team_id': ObjectId('67d5ef17e936e1912e2b2dd4')}
{'image': 'https://a.espncdn.com/i/headshots/nba/players/full/4433620.png', 'name': 'Emoni Bates', 'number': '21', 'position': 'A', 'age': '21', 'height': '2.03 m', 'weight': '77 kg', 'university': 'Eastern Michigan', 'salary': '--', 'team_id': ObjectId('67d5ef17e936e1912e2b2dd4')}
{'image': 'https://a.espncdn.com/i/headshots/nba/players/full/4396907.png', 'name': 'Darius Garland', 'number': '10', 'position': 'PG', 'age': '25', 'height': '1.85 m', 'weight': '87 kg', 'university': 'Vanderbilt', 'salary': '$36,725,670', 'team_id': ObjectId('67d5ef17e936e1912e2b2dd4')}
{'image': 'https://a.espncdn.com/i/headshots/nba/players/full/2596112.png', 'name': 'Javonte Green', 'number': '8', 'position': 'G', 'age': '31

### Conectando ao MongoDB

In [ ]:
from pymongo import MongoClient
import os
from dotenv import load_dotenv

# Carregando as variáveis de ambiente
load_dotenv()

user = os.getenv('MONGO_USER')
password = os.getenv('MONGO_PASSWORD')
cluster = os.getenv('MONGO_CLUSTER')

# Conectar ao MongoDB 
client = MongoClient(f"mongodb+srv://{user}:{password}@{cluster}/?retryWrites=true&w=majority&appName=Cluster0")
db = client["nbaBd"]  # Substitua pelo nome do seu banco de dados

# Coleções
teams_collection = db["teams"]  # Coleção de times
players_collection = db["players"]  # Coleção de jogadores


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


### Inserindo os dados

In [ ]:
result_teams = teams_collection.insert_many(list_teams)

In [30]:
result_players = players_collection.insert_many(list_players)